In [2]:
import heapq

class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.

    Attributes
    ----------
    <include your list and description here>

    '''
    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        your docstring
        Parameters
        ----------
        other
            <include your description here>

        Returns
        -------
        bool
            <include your description here>
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        your docstring
        Parameters
        ----------
        schedule
            <include your description here>
        '''
        self.function(schedule, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.

    The `now` attribute contains the time at which the last event was run.

    Attributes
    ----------
    <include your list and description here>

    '''

    def __init__(self):
        self.now = 0
        self.priority_queue = []

    def add_event_at(self, timestamp, function, *args, **kwargs):
        '''
        your docstring
        Parameters
        ----------
        <include your list and description here>

        Returns
        -------
        <include your list and description here>
        '''
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))

    def add_event_after(self, interval, function, *args, **kwargs):
        '''
        your docstring
        Parameters
        ----------
        <include your list and description here>

        Returns
        -------
        <include your list and description here>
        '''
        self.add_event_at(self.now + interval, function, *args, **kwargs)

    def next_event_time(self):
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        '''
        your docstring
        Parameters
        ----------
        <include your list and description here>

        Returns
        -------
        <include your list and description here>
        '''
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)

    def __repr__(self):
        return (
            f'Schedule() at time {self.now}min ' +
            f'with {len(self.priority_queue)} events in the queue')

    def print_events(self):
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'  ⏱ {event.timestamp}min: {event.function.__name__}')

In [3]:
# CHANGES:
# We have swapped Queue_MD1 with Queue_MG1 (only the name)
# We have swapped GroceryStore_MD1 with GroceryStore_MG1 (only the name)
# We have swapped run_simulation_MD1 with run_simulation_MG1 (only the name)
import scipy.stats as sts
class Queue_MG1:
    def __init__(self, service_distribution):
        # Store the deterministic service time for an M/D/1 queue
        self.service_distribution = service_distribution
        # We start with an empty queue and the server not busy
        self.people_in_queue = 0
        self.people_being_served = 0

    def add_customer(self, schedule):
        # Add the customer to the queue
        self.people_in_queue += 1
        print(
            f'⏱{schedule.now:5.2f}min: Add customer to queue.  '
            f' 🧍🏾‍♀️People in the queue: {self.people_in_queue}')
        if self.people_being_served < 1:
            # This customer can be served immediately
            schedule.add_event_after(0, self.start_serving_customer)

    def start_serving_customer(self, schedule):
        # Move the customer from the queue to a server
        self.people_in_queue -= 1
        self.people_being_served += 1
        print(
            f'⏱{schedule.now:5.2f}min: Start serving customer. '
            f' 🧍🏾‍♀️People in the queue: {self.people_in_queue}')
        # Schedule when the server will be done with the customer
        schedule.add_event_after(
            self.service_distribution.rvs(),
            self.finish_serving_customer)

    def finish_serving_customer(self, schedule):
        # Remove the customer from the server
        self.people_being_served -= 1
        print(
            f'⏱{ schedule.now:5.2f}min: Stop serving customer.  '
            f' 🧍🏾‍♀️People in the queue: {self.people_in_queue}')
        if self.people_in_queue > 0:
            # There are more people in the queue so serve the next customer
            schedule.add_event_after(0, self.start_serving_customer)


class GroceryStore_MG1:
    def __init__(self, arrival_distribution, service_distribution):
        self.queue = Queue_MG1(service_distribution)
        self.arrival_distribution = arrival_distribution

    def add_customer(self, schedule):
        # Add this customer to the queue
        self.queue.add_customer(schedule)
        # Schedule when to add another customer
        schedule.add_event_after(
            self.arrival_distribution.rvs(),
            self.add_customer)

    def run(self, schedule):
        # Schedule when the first customer arrives
        schedule.add_event_after(
            self.arrival_distribution.rvs(),
            self.add_customer)


def run_simulation_MG1(arrival_distribution, service_distribution, run_until):
    schedule = Schedule()
    grocery_store = GroceryStore_MG1(arrival_distribution, service_distribution)
    grocery_store.run(schedule)
    while schedule.next_event_time() < run_until:
        schedule.run_next_event()
    return grocery_store

In [4]:
# Run the simulation and record the time and queue length
# (you can take inspiration from code cell 3)
import numpy as np

# Exponential with lambda = 1.2
arrival_distribution = sts.expon(scale = 1/1.2) # your code here

# Normal Distribution with mu = 3 and sigma = 1 (standard deviation)
service_time_mean = 3# your code here
service_time_variance  = 1**2# your code here
service_distribution = sts.norm(loc = service_time_mean,
                               scale = np.sqrt(service_time_variance))# your code here
duration = 5 # in minutes

## run the actual simulation
grocery_store = run_simulation_MG1(arrival_distribution, service_distribution, duration)# your code here

##  your comment here about what the next line does
print(f'\n🔚 At closing time, there are {grocery_store.queue.people_in_queue} people in the queue.')

⏱ 0.15min: Add customer to queue.   🧍🏾‍♀️People in the queue: 1
⏱ 0.15min: Start serving customer.  🧍🏾‍♀️People in the queue: 0
⏱ 0.75min: Add customer to queue.   🧍🏾‍♀️People in the queue: 1
⏱ 0.79min: Add customer to queue.   🧍🏾‍♀️People in the queue: 2
⏱ 2.36min: Add customer to queue.   🧍🏾‍♀️People in the queue: 3
⏱ 2.52min: Stop serving customer.   🧍🏾‍♀️People in the queue: 3
⏱ 2.52min: Start serving customer.  🧍🏾‍♀️People in the queue: 2
⏱ 2.81min: Add customer to queue.   🧍🏾‍♀️People in the queue: 3
⏱ 3.01min: Add customer to queue.   🧍🏾‍♀️People in the queue: 4
⏱ 3.62min: Add customer to queue.   🧍🏾‍♀️People in the queue: 5
⏱ 4.47min: Add customer to queue.   🧍🏾‍♀️People in the queue: 6
⏱ 4.65min: Add customer to queue.   🧍🏾‍♀️People in the queue: 7

🔚 At closing time, there are 7 people in the queue.


In [15]:
# Run the simulation and record the time and queue length

# Exponential with lambda = 1.2
arrival_distribution = sts.expon(scale = 1/1.2) # your code here
service_distribution = sts.norm(loc = service_time_mean,
                               scale = np.sqrt(service_time_variance))# your code here
duration = 5

## run the actual simulation
grocery_store = run_simulation_MG1(arrival_distribution, service_distribution, duration)



⏱ 1.68min: Add customer to queue.   🧍🏾‍♀️People in the queue: 1
⏱ 1.68min: Start serving customer.  🧍🏾‍♀️People in the queue: 0
⏱ 1.80min: Add customer to queue.   🧍🏾‍♀️People in the queue: 1
⏱ 2.24min: Add customer to queue.   🧍🏾‍♀️People in the queue: 2
⏱ 2.44min: Add customer to queue.   🧍🏾‍♀️People in the queue: 3
⏱ 2.79min: Add customer to queue.   🧍🏾‍♀️People in the queue: 4
⏱ 3.01min: Stop serving customer.   🧍🏾‍♀️People in the queue: 4
⏱ 3.01min: Start serving customer.  🧍🏾‍♀️People in the queue: 3
⏱ 3.96min: Add customer to queue.   🧍🏾‍♀️People in the queue: 4
⏱ 4.05min: Add customer to queue.   🧍🏾‍♀️People in the queue: 5
⏱ 4.43min: Add customer to queue.   🧍🏾‍♀️People in the queue: 6
⏱ 4.50min: Add customer to queue.   🧍🏾‍♀️People in the queue: 7
⏱ 4.85min: Add customer to queue.   🧍🏾‍♀️People in the queue: 8
